In [1]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
from langchain_chroma import Chroma
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
from math import ceil
from tqdm import tqdm

In [2]:
load_dotenv()

True

In [3]:
split_docs_df = pd.read_csv('../data/split_docs_20250909.csv')
print(split_docs_df.shape)
split_docs_df.head()

(299, 1)


,docs
0,보고서 목적 및 적용 범위
1,본 보고서는 가상의 웹 애플리케이션 개발 회사인 주식회사 CoreFlow의 체계적이...
2,"이 규정집은 단순히 법적 의무를 명시하는 서류철을 넘어, CoreFlow의 모든 구..."
3,"사내 규정은 명확한 규칙과 절차를 확립함으로써 회사 운영의 일관성을 유지하고, 불필..."
4,"특히 IT 산업의 급변하는 환경 속에서, 본 규정집은 인재 관리, 지식재산권 보호,..."


In [4]:
texts_list = split_docs_df["docs"].tolist()
texts_list[:5]

['보고서 목적 및 적용 범위',
 '본 보고서는 가상의 웹 애플리케이션 개발 회사인 주식회사 CoreFlow의 체계적이고 투명한 운영을 위한 사내 규정을 총망라하여 제공한다',
 '이 규정집은 단순히 법적 의무를 명시하는 서류철을 넘어, CoreFlow의 모든 구성원들이 회사의 비전과 가치에 따라 행동하고, 법률 및 윤리적 기준을 준수하며, 궁극적으로 회사의 지속 가능한 성장을 도모할 수 있도록 돕는 실무적 가이드라인 역할을 한다',
 '사내 규정은 명확한 규칙과 절차를 확립함으로써 회사 운영의 일관성을 유지하고, 불필요한 혼란과 갈등을 줄이는 데 필수적이다',
 '특히 IT 산업의 급변하는 환경 속에서, 본 규정집은 인재 관리, 지식재산권 보호, 정보 보안 등 핵심적인 영역에서 발생하는 다양한 문제를 사전에 예방하고 해결하는 데 중요한 역할을 수행할 것이다']

In [5]:
headers = {
    "authorization": os.getenv("myserver_api_key")
}

embed_url = os.getenv("myserver_url")+":8000/embed"
rerank_url = os.getenv("myserver_url")+":8000/rerank"

In [6]:
class CustomEmbeddings:
    def embed_documents(self, texts):
        payload = {
            "texts": texts,
            "is_query": False
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"]
    
    def embed_query(self, text):
        payload = {
            "texts": [text],
            "is_query": True
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"][0]

In [7]:
embeddings = CustomEmbeddings()

In [8]:
def batched(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

In [9]:
for chunk in tqdm(batched(texts_list, 16)):  # 32~128 사이로 맞춰봐
    vector_store = Chroma.from_texts(
        texts=chunk, embedding=embeddings, persist_directory="../chroma_db"
    )

19it [01:58,  6.24s/it]
